In [1]:
import numpy as np
import cv2 as cv
import pandas as pd
import tensorflow as tf
import os
import caer
import canaro
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [2]:
IMG_SIZE = (80,80)
CHANNELS = 1
BASE_PATH = "Faces/simpsons.dataset/simpsons_dataset"
BATCH_SIZE = 32
EPOCHS = 10
characters = os.listdir(BASE_PATH)
number_of_images = {}

for character in characters:
    number_of_images[character] = len(os.listdir(os.path.join(BASE_PATH,character)))

number_of_images = caer.sort_dict(number_of_images, descending=True)

characters = []
for i in number_of_images[0:10]:
    characters.append(i[0])

train = caer.preprocess_from_dir(BASE_PATH,characters,channels=CHANNELS,IMG_SIZE=IMG_SIZE, isShuffle=True)


[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.004451274871826172s
[INFO] At 1000 files
[INFO] 1454 files found in 0.0s
[INFO] At 1000 files
[INFO] 1452 files found in 0.0s
[INFO] At 1000 files
[INFO] 1354 files found in 0.0s
[INFO] At 1000 files
[INFO] 1342 files found in 0.0s
[INFO] At 1000 files
[INFO] 1291 files found in 0.0s
[INFO] At 1000 files
[INFO] 1206 files found in 0.0s
[INFO] At 1000 files
[INFO] 1194 files found in 0.001374959945678711s
[INFO] At 1000 files
[INFO] 1193 files found in 0.0s
[INFO] At 1000 files
[INFO] 1079 files found in 0.016206979751586914s
----------------------------------------------
[INFO] 13811 files preprocessed! Took 0m 22s


In [3]:
features, labels = caer.sep_train(train,IMG_SIZE=IMG_SIZE)

# Normalise The dataset
features = caer.normalize(features)
labels = tf.keras.utils.to_categorical(labels,len(characters))

# Break into train and validation
x_train, x_val, y_train, y_val = caer.train_val_split(features,labels, val_ratio=0.2)


In [4]:
# Image data generator (introduces randomness in network ==> better accuracy)
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

# Create our model (returns the compiled model)
w, h = IMG_SIZE[:2]

model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=CHANNELS, output_dim=len(characters), 
                                         loss='binary_crossentropy', decay=1e-7, learning_rate=0.001, momentum=0.9,
                                         nesterov=True)

model.summary()

# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(w, h, CHANNELS)))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(256, (3, 3), padding='same', activation='relu')) 
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))

# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(w, h,CHANNELS)))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(256, (3, 3), padding='same', activation='relu')) 
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(1024, activation='relu'))

# # Output Layer
# model.add(Dense(len(characters), activation='softmax'))

# optimizer = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# model.summary()


ValueError: decay is deprecated in the new Keras optimizer, please check the docstring for valid arguments, or use the legacy optimizer, e.g., tf.keras.optimizers.legacy.SGD.

In [ ]:

my_callbacks = [
    tf.keras.callbacks.LearningRateScheduler(canaro.lr_schedule)
]
training = model.fit(train_gen,steps_per_epoch=len(x_train)//BATCH_SIZE,epochs=EPOCHS,validation_data=(x_val, y_val), validation_steps=len(x_val)//BATCH_SIZE,callbacks=my_callbacks)

Epoch 1/10
345/345 [==============================] - ETA: 0s - loss: 0.3424 - accuracy: 0.1548WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 86 batches). You may need to use the repeat() function when building your dataset.


345/345 [==============================] - 66s 191ms/step - loss: 0.3424 - accuracy: 0.1548 - val_loss: 0.3330 - val_accuracy: 0.1574 - lr: 0.0100
Epoch 2/10
 23/345 [=>............................] - ETA: 1:01 - loss: 0.3234 - accuracy: 0.1603

KeyboardInterrupt: 

In [ ]:
# characters_tt = []
# characters_tt.append(characters[0])
# features = []
# labels = []
# index = 0
# for character in characters_tt:
#     print(character)
#     path = os.path.join(BASE_PATH,character)
#     for file in os.listdir(path):
#         print(file)
#         img = cv.imread(os.path.join(path,file))
#         gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
#         gray_img = cv.resize(gray_img, IMG_SIZE)
#         features.append(gray_img)
#         labels.append(index)
#     index += 1
# permutations = np.random.permutation(len(labels))    
# features = features[permutations]
# labels = labels[permutations]
    
